# Detecting bad comments using BERT

In this notebook, we build a classifier using [BERT](https://arxiv.org/abs/1810.04805), a pretrained NLP model that can be used for transfer learning on text data.  We will use the [*ktrain* library](https://github.com/amaiya/ktrain), a lightweight wrapper around Keras to help train (and deploy) neural networks.  For more information on *ktrain*, see [this Medium post](https://towardsdatascience.com/ktrain-a-lightweight-wrapper-for-keras-to-help-train-neural-networks-82851ba889c).

We will begin by installing *ktrain* and importing the required *ktrain* modules.

In [2]:
# install ktrain
# !pip3 install ktrain

In [3]:
# install eli5

# !pip3 install git+https://github.com/amaiya/eli5@tfkeras_0_10_1

In [ ]:
# import ktrain
import ktrain
from ktrain import text

In [2]:
ktrain.__version__

'0.7.2'

In [ ]:
import numpy as np
import pandas as pd

In [7]:
df = pd.read_csv("test_data.csv")

In [8]:
df.head()

,comment,language,translation,harmful,harmful_type
0,"Эх, была б такая страна, а так придётся перепл...",ru,"Oh, there was such a country, and you have to ...",1,7
1,Jajajaja para eso se las compro al top manta,es,Hahahaha for that I buy them the top blanket,0,0
2,idomu ar cia fake,lt,This is a fake one,1,7
3,Usenme como botón de dislike,gl,Use me as a dislike button,1,7
4,Можна замовити?,uk,Can I order?,0,0


In [9]:
df = df.loc[:, ['translation', 'harmful']]
df.columns = ['text', 'label']

In [10]:
df.head(3)

,text,label
0,"Oh, there was such a country, and you have to ...",1
1,Hahahaha for that I buy them the top blanket,0
2,This is a fake one,1


In [11]:
# drop missing values
df = df.dropna(how="any")
print(df.shape)

(28076, 2)


## STEP 1:  Load and Preprocess the Dataset

In [12]:
# split datasets
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(
    df.text, df.label, test_size=0.20, random_state=42
)

In [13]:
_x_train = [x for x in x_train]
_y_train = [str(x) for x in y_train]
_x_test = [x for x in x_test]
_y_test = [str(x) for x in y_test]

In [14]:
# classes names

classes_names = np.unique(df.label)
classes_names_str = [str(name) for name in classes_names]

In [15]:
classes_names_str

['0', '1']

In [16]:
NUM_WORDS = 5000
MAXLEN = 150

(x_train, y_train), (x_test, y_test), preproc = text.texts_from_array(x_train=_x_train, y_train=np.array(_y_train),
                                                                       x_test=_x_test, y_test=np.array(_y_test),
                                                                      class_names=classes_names_str,
                                                                       #maxlen=500,
                                                                      max_features = NUM_WORDS,
                                                                       maxlen=MAXLEN,
                                                                      ngram_range=1,
                                                                       preprocess_mode='bert')



preprocessing train...
language: en


preprocessing test...
language: en


## STEP 2:  Load a pretrained BERT model and wrap it in a `ktrain.Learner` object

This step can be condensed into a single line of code, but we execute it as two lines for clarity. (You can ignore the deprecation warnings arising from Keras 2.2.4 with TensorFlow 1.14.0.)  

In [25]:
text.print_text_classifiers()

fasttext: a fastText-like model (http://arxiv.org/pdf/1607.01759.pdf)
logreg: logistic regression using a trainable Embedding layer
nbsvm: NBSVM model (http://www.aclweb.org/anthology/P12-2018)
bigru: Bidirectional GRU with pretrained word vectors (https://arxiv.org/abs/1712.09405)
standard_gru: simple 2-layer GRU with randomly initialized embeddings
bert: Bidirectional Encoder Representations from Transformers (https://arxiv.org/abs/1810.04805)


In [26]:
model = text.text_classifier('bert', (x_train, np.array(y_train)), preproc=preproc)
#learner = ktrain.get_learner(model,train_data=(x_train, y_train), val_data=(x_test, y_test), batch_size=6)
learner = ktrain.get_learner(model,train_data=(x_train, np.array(y_train)), val_data=(x_test, np.array(y_test)), batch_size=3)

Is Multi-Label? False
maxlen is 150
done.


In [0]:
 #learning rate finder to find a good learning rate
 #learner.lr_find(max_epochs=8)
 #learner.lr_plot()

In [0]:
#learner.autofit(..., 3)

## STEP 3:  Train and Fine-Tune the Model

We employ the `learner.fit_onecycle` method in *ktrain* that employs the use of a [1cycle learning  rate schedule](https://arxiv.org/pdf/1803.09820.pdf).  We use a learning rate of 2e-5 based on recommendations from [the original paper](https://arxiv.org/abs/1810.04805).

In [30]:
learner.fit_onecycle(2e-5, 3)



begin training using onecycle policy with max lr of 2e-05...
Train on 22460 samples, validate on 5616 samples
Epoch 1/3
22460/22460 [==============================] - 1083s 48ms/sample - loss: 0.3817 - acc: 0.8173 - val_loss: 0.3291 - val_acc: 0.8508
Epoch 2/3
22460/22460 [==============================] - 1082s 48ms/sample - loss: 0.2956 - acc: 0.8709 - val_loss: 0.3159 - val_acc: 0.8494
Epoch 3/3
22460/22460 [==============================] - 1082s 48ms/sample - loss: 0.1591 - acc: 0.9379 - val_loss: 0.3740 - val_acc: 0.8615


In [31]:
learner.validate(val_data=(x_test, y_test), class_names=classes_names_str)

              precision    recall  f1-score   support

           0       0.87      0.89      0.88      3203
           1       0.85      0.83      0.84      2413

    accuracy                           0.86      5616
   macro avg       0.86      0.86      0.86      5616
weighted avg       0.86      0.86      0.86      5616



array([[2845,  358],
       [ 420, 1993]])

In [32]:
# invoke view_top_losses to view the most misclassified review in the validation set

learner.view_top_losses(n=1, preproc=preproc)

----------
id:3932 | loss:8.82 | true:1 | pred:0)

[CLS] and how much is the type ? [UNK] [SEP]


In [0]:
predictor = ktrain.get_predictor(learner.model, preproc)

In [0]:
#predictor.save('bert_model')

In [0]:
# predictor = ktrain.load_predictor('bert_model')

## explainability

In [0]:
doc = _x_test[38]

In [48]:

predictor.explain(doc)


## Making predictions



In [49]:
# correctly predict a toxic comment
predictor.predict(["If you don't stop immediately, I will kill you."])

['1']

In [52]:
predictor.explain("If you don't stop immediately, I will kill you.")

In [50]:
predictor.predict(["Okay - I'm calling it a night. See you tomorrow."])

['0']

In [53]:
predictor.explain("Okay - I'm calling it a night. See you tomorrow.")

In [55]:
predictor.explain("Bad night here. But loved this product!")

In [57]:
predictor.explain("Good night here. But hated this product!")